# Spending patterns exposed: relating country differences in budget expenditure to socio-economic indicators

#### Summary
All sovereign countries spread their public expenditure among different broad categories, like defence, health and education. However, each country may spend this money differently, prioritizing some budget items over others. The present project investigates whether we can group countries based on the way they allocate their budget expenditure, and whether those groups are meaningful in terms of predicting some socio-economic indicators. We tackled this problem by using machine learning techniques. While initial k-means clustering analysis did not give rise to meaningful expenditure-based country clusters, subsequent Principal Component Analysis (PCA) found that most budget expenditure variance could be explained by a reduced number of axes. The most important of them captured a trend where those countries that spent more on health and social protection also spent less on basic bureaucratic services. The value of each country on such principal component was highly correlated with its Human Development Index (HDI) value, and moderately correlated with the proportion of public employees over the total workforce. Specifically, countries spending more on health and social protection tended to be more developed and possessed more public employees. Overall, our results support that some socio-economic indicators can indeed be predicted by the way countries spend their budget.

## 1. Load libraries
We will start by importing the libraries necessary for this project. Numpy, scipy.stats and pandas contain a series of functions to deal with mathematical calculations and data wrangling. To make plots, we will use plotly, and in particular its graphical objects, which allow for higher customization, even if at the expense of more lines of code. Finally, to undergo the machine learning-relevant part we will use a few functions from scikit-learn. Since this library is composed of several modules with a lot of functions that would take a lot of resources to load, we will import the necessary functions individually from their respective modules.

In [9]:
#import whole libraries
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

#import individual functions of various modules within scikit-learn
from sklearn.cluster import KMeans #necessary for k-means clustering
from sklearn.metrics import silhouette_samples, silhouette_score #necessary for the silhouette method of optimal k
from sklearn.decomposition import PCA #necessary for Principal Component Analysis

## 2. Define graphical parameters
Next we will set a series of display preferences for our data. We will also predefine a custom theme and palette for our figures.

In [10]:
#avoid scientific notation and show numeric values rounded to two decimals
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#set max rows and columns when displaying data
pd.set_option('display.max_columns', None) #no maximum number of columns
pd.set_option('display.max_rows', 25) #maximum number of rows

#set font size
font_size_regular = 12

#create the layout for figures with a custom theme
mylayout = go.Layout(
    xaxis=dict(
        linecolor='black',
        gridcolor='lightgrey',
        tickfont=dict(size=font_size_regular - 2),
        zeroline=True,
        zerolinecolor='lightgrey',
        zerolinewidth=1
    ),
    yaxis=dict(
        linecolor='black',
        gridcolor='lightgrey',
        tickfont=dict(size=font_size_regular - 2),
        zeroline=True,
        zerolinecolor='lightgrey',
        zerolinewidth=1
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    font=dict(size=font_size_regular),
)

#set plot sizes
plot_size_shortwidth = 800 #width of a figure taking the whole width of the page, in px
plot_size_shortheight = 500 #height of a one-row figure, in px

#defining a color palette for our figures involving qualitative categories
palette_qual = ['#04BB85','#8F76D7','#FF9500','#019BBF','#F62F36','#9D5E31','#E65898','#B1C019','#FDC903','#C1A471'] 

## 3. Preparing the country budgets dataset
The bulk of our analyses will employ [a dataset from the United Nations data portal](https://data.un.org/Data.aspx?d=SNA&f=group_code%3A301) that includes information on how the public expenditure of several countries is assigned to different budget items, like health, defence and education. In this section we will load the dataset and wrangle the data in order to be able to apply a series of visualizations and machine learning techniques later on.

### 3.1. Loading the dataset
We will first load the dataset, which was downloaded from the UN data portal as a .csv file.

In [11]:
#load the dataset as DataFrame
df_budget = pd.read_csv('UNdata_Export_20240524_085932284.csv')

#display the DataFrame
display(df_budget)

,Country or Area,SNA93 Table Code,Sub Group,Item,SNA93 Item Code,Year,Series,Currency,SNA System,Fiscal Year Type,Value,Value Footnotes
0,Algeria,3.1,NaN,General public services,01,1976.00,10.00,Algerian dinar,1968.00,Western calendar year,1471000000.00,NaN
1,Algeria,3.1,NaN,Defence,02,1976.00,10.00,Algerian dinar,1968.00,Western calendar year,1632000000.00,NaN
2,Algeria,3.1,NaN,Economic affairs,04,1976.00,10.00,Algerian dinar,1968.00,Western calendar year,332000000.00,NaN
3,Algeria,3.1,NaN,Housing and community amenities,06,1976.00,10.00,Algerian dinar,1968.00,Western calendar year,0.00,NaN
4,Algeria,3.1,NaN,Health,07,1976.00,10.00,Algerian dinar,1968.00,Western calendar year,63000000.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
55764,100,Refers to transactions in military equipment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55765,101,"Includes recreational, cultural and sporting a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55766,102,Discrepancy between components and total as on...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55767,103,Totals exclude some expenditure that are colle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Most of the rows of the DataFrame displayed above include information on budget expenditure for a particular country, item and year. However, some rows are dedicated to footnotes that do not form part of our core data. Moreover, there are a few columns we are not interested in and could delete, as well as some columns that could use more intuitive names. Not to say we have not checked for redundant or missing data yet... Let's get to it then.

### 3.2. Wrangling the data to obtain a tidy DataFrame
The first steps will be selecting the portion of the DataFrame including the data we can run analyses with, sticking to the columns we really want, and giving them a simple and memorable name.

In [12]:
#filter data frame to include only rows with data, not including footnotes
df_budget = df_budget.iloc[:55664]

#select only relevant columns by their position
df_budget = df_budget.iloc[:, [0, 3, 4, 5, 10]]

#rename columns
df_budget = df_budget.set_axis(['country', 'item', 'item_code', 'year','value'], axis=1)

#display DataFrame
display(df_budget)

#display type of each column
df_budget.dtypes

,country,item,item_code,year,value
0,Algeria,General public services,01,1976.00,1471000000.00
1,Algeria,Defence,02,1976.00,1632000000.00
2,Algeria,Economic affairs,04,1976.00,332000000.00
3,Algeria,Housing and community amenities,06,1976.00,0.00
4,Algeria,Health,07,1976.00,63000000.00
...,...,...,...,...,...
55659,Zimbabwe,Health,07,1970.00,13000000.00
55660,Zimbabwe,"Recreation, culture and religion",08,1970.00,3000000.00
55661,Zimbabwe,Education,09,1970.00,17000000.00
55662,Zimbabwe,Plus: (Other functions),NaN,1970.00,-3000000.00


country       object
item          object
item_code     object
year         float64
value        float64
dtype: object

This looks way cleaner. Each row of the DataFrame carries information on the amount of money devoted by a country to a specific budget item on a specific year. This is contained in the following columns:

- `country`: string (object) column with the name of the country.
- `item`:  string column specifying the area of budget expenditure.
- `item_code`: string column with the numeric code associated with the budget item.
- `year`: numeric column with the year.
- `value`: numeric column with the amount spent, in the local currency for the country.

Yes, an obvious problem is that the `value` column will have values in multiple currencies. Let's not worry about that yet. To understand the different budget items, let's take a look at the unique values the combination of `item` and `item_code` can take. This will also help us see if those columns have a univocal mapping.

In [13]:
#print unique combinations of item and item_code and sort them
display(df_budget[['item', 'item_code']].drop_duplicates().sort_values(by='item_code'))

,item,item_code
0,General public services,01
1,Defence,02
41,Public order and safety,03
2,Economic affairs,04
43,Environment protection,05
3,Housing and community amenities,06
4,Health,07
5,"Recreation, culture and religion",08
6,Education,09
7,Social protection,10


Good, looks like we have only one mapping. Items associated to codes 01 to 10 follow the classification of the functions of government by the Organisation for Economic Co-operation and Development. This is how the [Eurostat website](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Classification_of_the_functions_of_government_(COFOG)) defines each item:

- General public services: Executive and legislative organs, financial and fiscal affairs, external affairs; foreign economic aid; general services; basic research; R&D related to general public services; general public services n.e.c.; public debt transactions, transfers of a general character between different levels of government.
- Defence:	Military defence; civil defence; foreign military aid, R&D related to defence; defence n.e.c.
- Public order and safety:	Police services; fire-protection services; law courts; prisons; R&D related to public order and safety; public order and safety n.e.c.
- Economic affairs:	General economic, commercial and labour affairs; agriculture, forestry; fishing and hunting; fuel and energy; mining, manufacturing and construction; transport; communication; other industries, R&D related to economic affairs; economic affairs n.e.c.
- Environmental protection:	Waste management; water waste management; pollution abatement; protection of biodiversity and landscape; R&D related to environmental protection.
- Housing and community amenities:	Housing development; community development; water supply; street lighting; R&D related to housing and community amenities; housing and community amenities n.e.c.
- Health:	Medical products, appliances and equipment; outpatient services; hospital services; public health services; R&D related to health; health n.e.c.
- Recreation, culture and religion:	Recreational and sporting services; cultural services; broadcasting and publishing services; religious and other community services, R&D related to recreation, culture and religion; recreation; culture and religion n.e.c.
- Education:	Pre-primary, primary, secondary and tertiary education, post-secondary non-tertiary education, education non definable by level, subsidiary services to education, R&D; n.e.c.
- Social protection:	Sickness and disability; old age; survivors; family and children; unemployment; housing; R&D; social protection and social exclusion n.e.c.

The other values `item` can take is *Plus: (Other functions)* and *Equals: General government final consumption expenditure*. The first refers to other functions, while the second conveys the sum of all individual items. We have no information on what this *other functions* specifically represents. Across countries, this could mean something very different and lead to biases when conducting our clustering. To avoid confusion we will delete those rows from the DataFrame. We will also get rid of the final expenditure rows, as we will calculate it ourselves by summing the budget over the 10 items we keep.

In [14]:
# remove rows containing other items and total expenditure
df_budget = df_budget[df_budget['item'] != 'Plus: (Other functions)']
df_budget = df_budget[df_budget['item'] != 'Equals: General government final consumption expenditure']

Another classic problem in public datasets is that we often find duplicate rows. Since the amount specified in `value` could be slightly different, we will identify duplicates by finding the rows where `country`, `item_code` and `year` are the same. Then we will compare it to the number of rows of the DataFrame we have now.

In [15]:
#display rows that share the same country, item_code and year information (partial duplicates)
display(df_budget[df_budget.duplicated(subset=['country', 'item_code', 'year'], keep=False)])

#show number of rows of the DataFrame
len(df_budget)

,country,item,item_code,year,value
80,Andorra,General public services,01,2018.00,116127786.77
81,Andorra,General public services,01,2018.00,128666000.00
82,Andorra,Public order and safety,03,2018.00,42150348.94
83,Andorra,Public order and safety,03,2018.00,36931000.00
84,Andorra,Economic affairs,04,2018.00,35916078.50
...,...,...,...,...,...
55286,Yemen,"Recreation, culture and religion",08,2000.00,1206000000.00
55287,Yemen,Education,09,2000.00,71456000000.00
55288,Yemen,Education,09,2000.00,68746000000.00
55289,Yemen,Social protection,10,2000.00,367000000.00


46613

Oh no, 27286 rows of the total 46613 are redundant. We will keep the first unique ones and drop the rest.

In [16]:
#drop partial duplicates, keeping the first occurrence
df_budget = df_budget.drop_duplicates(subset=['country', 'item_code', 'year'], keep='first')

#again, display partial duplicate rows
display(df_budget[df_budget.duplicated(subset=['country', 'item_code', 'year'], keep=False)])

,country,item,item_code,year,value


By the output of the last line above, we can see no more rows are duplicates. Now on to a more difficult problem: missing data. Even if we have data for different years, ideally we would like to use data from a same, recent year for all countries. But it is very likely all countries will not have data for the same years. On top of that, the data may not exist for all of our 10 items. To see the extent to which this happens, we will create a function that displays, for the most recent years (2010 to 2022), the amount of countries we have data for, and how many countries have at least a given number of rows. The maximum a country could have is 10 rows, corresponding to our 10 items.

In [17]:
#define function that counts the number of countries with at least a certain number of rows
def count_countries_by_min_rows(df, min_rows_list, min_year, max_year):
    #filter to include data between min_year and max_year
    df_years = df[(df['year'] >= min_year) & (df['year'] <= max_year)]

    #group by year and count the number of unique countries for each year
    ncountries_per_year = df_years.groupby('year')['country'].nunique().reset_index()
    ncountries_per_year.columns = ['year', 'number_of_countries']

    #group by year and country to count the number of rows per country per year
    country_counts_per_year = df_years.groupby(['year', 'country']).size().reset_index(name='counts')

    #initialize the result dataframe with the number of unique countries per year to later append columns calculated below
    result = ncountries_per_year.copy()

    #run loop to calculate number of countries with at least the number of rows for each of the numbers of the list
    for min_rows in min_rows_list:
        #filter to include only  countries with at least min_rows
        countries_min_rows = country_counts_per_year[country_counts_per_year['counts'] >= min_rows]

        #group by year to count the number of countries with at least min_rows
        countries_min_rows_per_year = countries_min_rows.groupby('year')['country'].nunique().reset_index()
        countries_min_rows_per_year.columns = ['year', f'ncountries_{min_rows}rows']

        #merge with the result dataframe
        result = result.merge(countries_min_rows_per_year, on='year', how='left').fillna(0)

    return result

#run function
ncountries_peryear = count_countries_by_min_rows(df_budget, [8, 9, 10], 2010, 2022) #look into n countries with 8 to 10 rows, from 2010 to 2022

#display resulting DataFrame
display(ncountries_peryear)

,year,number_of_countries,ncountries_8rows,ncountries_9rows,ncountries_10rows
0,2010.00,85,67,66,54
1,2011.00,85,66,66,56
2,2012.00,82,64,64,54
3,2013.00,81,65,65,54
4,2014.00,80,63,63,54
5,2015.00,80,63,63,52
6,2016.00,81,63,63,52
7,2017.00,78,61,60,51
8,2018.00,76,61,60,53
9,2019.00,76,61,59,52


The generated DataFrame can reveal how much data we lose depending on how much we narrow down our dataset. First, by looking at the  `number_of_countries` column we see how we have data for less countries as years go by. So even if it would be tempting to use only data for the latest year, in doing so we would lose a lot of datapoints. By looking at the three rightmost columns, we can also appreciate that there are many countries that do not have data for all items. This is likely due to problems adscribing the expenditure to one of those 10 items. Whatever the cause may have been, for the sake of this project it may be better to play it safe and choose only countries and years for which data for all 10 items is present. What we will do is to choose all countries and years with 10 rows of data each, and stick to the latest year. This way, despite the information across countries provening from different years, we will be able to salvage as much data as possible while at the same time ensuring all items are represented, preventing our subsequent analyses from being biased due to categories being missing. 

In [18]:
#filter df keeping only data from years 2010 onwards
df_budget = df_budget[df_budget['year'] >= 2010]

#take only data where for each year each country has 10 rows of data (corresponding to expenditure for each item of budget) 
df_budget = df_budget[df_budget.groupby(['country', 'year'])['year'].transform('size') >= 10]

#for each country take data for latest available year
latest_year_data = df_budget.groupby('country')['year'].transform('max')
df_budget = df_budget[df_budget['year'] == latest_year_data].reset_index(drop=True)

#display the DataFrame
display(df_budget)

,country,item,item_code,year,value
0,Australia,General public services,01,2021.00,29382000000.00
1,Australia,Defence,02,2021.00,42718000000.00
2,Australia,Public order and safety,03,2021.00,38821000000.00
3,Australia,Economic affairs,04,2021.00,41182000000.00
4,Australia,Environment protection,05,2021.00,4792000000.00
...,...,...,...,...,...
585,Zambia,Housing and community amenities,06,2021.00,363433352.05
586,Zambia,Health,07,2021.00,155703527.01
587,Zambia,"Recreation, culture and religion",08,2021.00,158776287.05
588,Zambia,Education,09,2021.00,806607293.74


We are left with 590 rows, which should mean that we have data for 59 countries. Let's make sure though.

In [19]:
#display number of unique countries
display(df_budget['country'].nunique())

#display unique number of rows for our countries
display(df_budget.groupby('country').size().unique())

59

array([10])

Okay, 59 countries and all have 10 rows. Now let's see the range of years our data spans.

In [20]:
#display unique years data is for
display(df_budget['year'].unique())

array([2021., 2011., 2014., 2020., 2018., 2022., 2013., 2012., 2019.])

We have data from 2011 through 2022. Quite a wide range, but at least we will have a sizeable number of countries. Since we are at it, let's take a look at the countries we still have.

In [21]:
#display unique countries
display(df_budget['country'].unique())

array(['Australia', 'Austria', 'Bangladesh', 'Belgium', 'Bermuda',
       'Bulgaria', 'Chile',
       'China, Hong Kong Special Administrative Region', 'Croatia',
       'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Eswatini',
       'Faroe Islands', 'Fiji', 'Finland', 'France', 'Germany', 'Greece',
       'Greenland', 'Guatemala', 'Hungary', 'Iceland', 'India',
       'Iran (Islamic Republic of)', 'Ireland', 'Israel', 'Italy',
       'Japan', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Lithuania',
       'Luxembourg', 'Malaysia', 'Malta', 'Mongolia', 'Netherlands',
       'Norway', 'Poland', 'Portugal', 'Republic of Korea', 'Romania',
       'Russian Federation', 'Senegal', 'Seychelles', 'Slovakia',
       'Slovenia', 'Spain', 'Sri Lanka', 'Sweden', 'Switzerland',
       'Tanzania - Mainland', 'Thailand', 'Ukraine', 'United Kingdom',
       'Yemen', 'Zambia'], dtype=object)

Alright. There are a few things to consider. First, we are left with a list of mainly developed countries. That makes sense if we think that we narrowed down our list to those countries for which the most exhaustive data is available. This may impact our analyses, but there is not much we can do about it. Second, some of those names above belong to non-sovereign territories. By definition, this means that those lack full authority, typically lacking such structures like, for example, an army. It may be wise to remove them from the list.

In [22]:
#remove non-sovereign states
non_sovereign_states = ['Bermuda', 'China, Hong Kong Special Administrative Region', 'Faroe Islands', 'Greenland']
df_budget = df_budget[~df_budget['country'].isin(non_sovereign_states)]

Some of the remaining countries appear here with their official but quite long names. We will rename the values in the `country` column so those countries are identified by the short, straightforward name by which people refer to them.

In [23]:
#replace country names to make them shorter
df_budget['country'] = df_budget['country'].replace('Iran (Islamic Republic of)', 'Iran')
df_budget['country'] = df_budget['country'].replace('Republic of Korea', 'South Korea')
df_budget['country'] = df_budget['country'].replace('Russian Federation', 'Russia')
df_budget['country'] = df_budget['country'].replace('Tanzania - Mainland', 'Tanzania')

And now, a crucial thing. We will take the expenditure per item per country, currently in local currency, and obtain the percentage it represents over the total budget we will previously calculate. Then we will print data for an example country to see how it looks.

In [24]:
#calculate the percentage of the budget assigned to each item
df_budget['total_budget'] = df_budget.groupby('country')['value'].transform('sum') #calculate total budget by summing expenditure for all items within countries
df_budget['perc_budget'] = (df_budget['value'] / df_budget['total_budget']) * 100 #divide total expenditure on each item over total budget and multiply by 100

#display data for one example country
display(df_budget[df_budget['country'] == 'Australia'])

,country,item,item_code,year,value,total_budget,perc_budget
0,Australia,General public services,01,2021.00,29382000000.00,501729000000.00,5.86
1,Australia,Defence,02,2021.00,42718000000.00,501729000000.00,8.51
2,Australia,Public order and safety,03,2021.00,38821000000.00,501729000000.00,7.74
3,Australia,Economic affairs,04,2021.00,41182000000.00,501729000000.00,8.21
4,Australia,Environment protection,05,2021.00,4792000000.00,501729000000.00,0.96
5,Australia,Housing and community amenities,06,2021.00,5033000000.00,501729000000.00,1.00
6,Australia,Health,07,2021.00,155910000000.00,501729000000.00,31.07
7,Australia,"Recreation, culture and religion",08,2021.00,10742000000.00,501729000000.00,2.14
8,Australia,Education,09,2021.00,91830000000.00,501729000000.00,18.30
9,Australia,Social protection,10,2021.00,81319000000.00,501729000000.00,16.21


Nice and clean. We now have the final DataFrame we will work with. Another useful thing to have is a list of the unique countries we are left with, as well as a list of the budget expenditure items. These will come out handy very soon, when we start taking slices of our DataFrame and creating figures.

In [25]:
#obtain an alphabetically-ordered list of the countries
unique_countries = sorted(df_budget['country'].unique())

#obtain a list with unique items
unique_items = df_budget['item'].unique()

And finally, before starting with the machine learning part, let's visualize how each country's budget is divided among items. We will do it by creating a bar plot, where each country will have 10 horizontally-stacked bars, one per item, each having a length corresponding to the percentage of budget spent that item.

In [26]:
#initialize the figure
fig = go.Figure()

#add bars for each item
for item, color in zip(unique_items, palette_qual):
    fig.add_trace(go.Bar(
        y=df_budget[df_budget['item'] == item]['country'],
        x=df_budget[df_budget['item'] == item]['perc_budget'],
        name=item,
        orientation='h',
        marker=dict(color=color),
        customdata=df_budget[df_budget['item'] == item]['item'],
        hovertemplate='<b>%{y}</b><br><b>%{customdata}</b>: %{x:.2f}<extra></extra>'
    ))

#customize layout
fig.update_layout(
    height=1200,
    barmode='stack',
    title='Percentage of budget spent per item',
    xaxis=dict(title='Percentage of each item over total country budget', range=[0, 100]),
    yaxis=dict(title='Country', autorange='reversed'),
    legend=dict(title='Item', traceorder='reversed'),
    plot_bgcolor='white'
)

#show the figure
fig.show()

There is a lot of information, but it is very useful to have it so we can go back to it whenever we want. At a first glance we can see that the lion's share of most countries is spent on health and education. On the other hand, the items with the lowest spending tend to be environment protection and recreation, culture and religion. We can also see how this is less so for some countries. For instance, Zambia at the very bottom dedicates much less to health than to general public services. It is now time to find out whether those differences are systematic.

## 4. Extracting insights from the country budgets dataset by using machine learning
As said above, this project aims at examining whether we can assign countries to different groups according to how they spend their public money across items. Then, we will to see whether these groups translate into differences in terms of a series of socio-economic indicators. The first step, then, is to try and find country clusters.

### 4.1. Exploring country clusters via k-means clustering

In order to try to assign countries to groups according to their budget expenditure, an obvious candidate algorithm is k-means clustering. To understand the process, we can imagine a country's percentages for the 10 budget items to form a set of coordinates in a 10-dimensional space. By minimizing the squared Euclidean distances of such coordinates, the k-means clustering algorithm will assign each country to the cluster whose centroid is closest, and will do so iteratively until results do not significantly change. However, k-means clustering requires us to pre-specify the number of clusters. This leads to the problem of identifying which is the optimal cluster size, which can be solved through different methods. The most common of those is the elbow method, which we will follow next. 

The chunk code below uses the *scikit-learn* function *KMeans* to run k-means clustering for each number of clusters from 2 to 10. For each cluster number, we will extract the sum of squared distances (SSE) of samples to their closest cluster center.

In [27]:
#pivot the DataFrame to have items as columns and countries as rows, the data format the KMeans function requires
df_budget_pivot = df_budget.pivot(index='country', columns='item', values='perc_budget').fillna(0)

#determine the optimal number of clusters, from 2 to 10, using the elbow method and assign information to DataFrame
sse = []
n_clusters = list(range(2, 11))
for k in n_clusters:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_budget_pivot) #fit k-means
    sse.append(kmeans.inertia_) #extract SSE

elbow_df = pd.DataFrame({'n_k': n_clusters, 'sse': sse})

#display the DataFrame
display(elbow_df)

,n_k,sse
0,2,12983.15
1,3,11568.51
2,4,8714.21
3,5,7947.26
4,6,7061.22
5,7,6379.75
6,8,5366.35
7,9,5061.39
8,10,4632.52


We now have a DataFrame with the SSE for each cluster size. However, we need to visualize the data to clearly see if there is an optimal number of clusters. Since in what follows we will create a series of very similar figures in structure, we may as well define a function that will save us some lines of code in the long run.

In [28]:
#define a function that creates a plot of dots connected by lines
def plot_line_and_scatter(df, x_col, y_col, x_title, y_title, hover_text, title):

    #line plot
    line = go.Scatter(
        x=df[x_col],
        y=df[y_col],
        mode='lines',
        name='Line plot',
        hoverinfo='none',
        showlegend=False, 
        line=dict(color='black')
    )

    #scatter plot
    scatter = go.Scatter(
        x=df[x_col],
        y=df[y_col],
        mode='markers',
        hoverinfo='text',
        text=hover_text,
        showlegend=False, 
        marker=dict(size=10, color='black', line=dict(color='white', width=1)),
    )

    #create the figure
    fig = go.Figure(data=[scatter, line], layout=mylayout)

    #customize layout
    fig.update_layout(
        width=plot_size_shortwidth,
        height=plot_size_shortheight,
        title=title,
        xaxis_title=x_title,
        yaxis_title=y_title,
        margin=dict(l=40, r=40, t=40, b=40)
    )

    #show the figure
    fig.show()

#plot the figure by first defining a series of input parameters and then passing them to the function
x_col = 'n_k'  #name of column for x-axis
y_col = 'sse'  #name of column for y-axis
x_title = 'Number of clusters'  #title for x-axis
y_title = 'Sum of squared distances'  #title for y-axis
hover_text = [f'{x_title}: {x_value:.2f}<br>{y_title}: {y_value:.2f}'
              for x_value, y_value in zip(elbow_df[x_col], elbow_df[y_col])] #plotly's graphical object syntax for hover text
title = 'Results of elbow method for optimal K' #plot title

plot_line_and_scatter(elbow_df, x_col, y_col, x_title, y_title, hover_text, title) #run the function to see the figure

What we have above is a plot of the SSE for each cluster size. The elbow method is a graphical way to pick the number of clusters by sticking to the value corresponding to the "elbow" of the curve: that is, where a sharp reduction in the slope is produced. In our case, although there is a big change for k = 4, there are other points where the slope changes significantly. In uncertain cases like ours, a recommendation is to apply another method, called the silhouette method.

The silhouette method involves calculating the -1 to 1 silhouette score, which measures how similar an object is to its own cluster (cohesion) compared to other clusters (separation). In our implementation, for cluster sizes 2 to 10, we will calculate the silhouette score for each cluster, and for the whole dataset. The size with the highest score for the whole dataset should be considered optimal, as long as two requisites are met: the score for each cluster should be bigger than that for the whole dataset, and all clusters should have similar sizes.

In order to do all the aforementioned checks, we will create a similar figure to the previous one, but now displaying the dataset's silhouette score as a function of cluster size. Moreover, we will print the value for these whole dataset silhouette scores right next to the individual cluster silhouette scores.

In [29]:
#initialize lists to store silhouette scores and cluster sizes
silhouette_scores = []
cluster_sizes = []

#iterate over different numbers of clusters
for k in range(2, 11):
    #initialize KMeans with k clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    #fit KMeans to the data
    kmeans.fit(df_budget_pivot)
    
    #calculate silhouette score for each sample
    sample_silhouette_values = silhouette_samples(df_budget_pivot, kmeans.labels_)
    
    #calculate average silhouette score for the entire dataset
    silhouette_avg = silhouette_score(df_budget_pivot, kmeans.labels_)
    silhouette_scores.append(silhouette_avg)
    
    #calculate the number of data points in each cluster
    cluster_sizes.append(pd.Series(kmeans.labels_).value_counts().sort_index().tolist())
    
    #store detailed silhouette scores for each cluster
    cluster_silhouette_scores = []
    for i in range(k):
        cluster_silhouette_avg = sample_silhouette_values[kmeans.labels_ == i].mean()
        cluster_silhouette_scores.append(cluster_silhouette_avg)
    
    #display results for current k
    print(f'For k = {k}:')
    print(f'  Average silhouette score for the whole dataset: {silhouette_avg:.4f}')
    for i, score in enumerate(cluster_silhouette_scores):
        print(f'  Average silhouette score for cluster {i}: {score:.4f}')
    print(f'  Number of data points in each cluster: {cluster_sizes[-1]}')
    print('')

silhouette_df = pd.DataFrame({'n_k': n_clusters, 'silhouette_score': silhouette_scores})

#plot the figure
x_col = 'n_k'  #name of column for x-axis
y_col = 'silhouette_score'  #name of column for y-axis
x_title = 'Number of clusters'  #title for x-axis
y_title = 'Silhouette score'  #title for y-axis
hover_text = [f'{x_title}: {x_value:.2f}<br>{y_title}: {y_value:.2f}'
              for x_value, y_value in zip(silhouette_df[x_col], silhouette_df[y_col])] #hover text template
title = 'Results of silhouette method for optimal K'

plot_line_and_scatter(silhouette_df, x_col, y_col, x_title, y_title, hover_text, title) #run function with parameters defined above

For k = 2:
  Average silhouette score for the whole dataset: 0.3288
  Average silhouette score for cluster 0: 0.0363
  Average silhouette score for cluster 1: 0.4959
  Number of data points in each cluster: [20, 35]

For k = 3:
  Average silhouette score for the whole dataset: 0.1746
  Average silhouette score for cluster 0: -0.0367
  Average silhouette score for cluster 1: 0.3023
  Average silhouette score for cluster 2: 0.2183
  Number of data points in each cluster: [17, 23, 15]

For k = 4:
  Average silhouette score for the whole dataset: 0.2299
  Average silhouette score for cluster 0: 0.0864
  Average silhouette score for cluster 1: 0.3868
  Average silhouette score for cluster 2: 0.1053
  Average silhouette score for cluster 3: 0.2392
  Number of data points in each cluster: [16, 24, 12, 3]

For k = 5:
  Average silhouette score for the whole dataset: 0.1571
  Average silhouette score for cluster 0: 0.0224
  Average silhouette score for cluster 1: 0.1076
  Average silhouette sco

Based on the whole dataset silhouette scores, we should pick 2 as the optimal cluster size. However, the first of the requisites is not fulfilled: the first cluster's silhouette score is smaller than that for the whole dataset. Actually, for no cluster size is this requirement met. It is possible, then, that our data does not lend itself to clear clustering. Instead, the across-country differences in budget expenditure may be better described as a continuum.

### 4.2. Reducing the complexity of multiple budget items via Principal Component Analysis

Even if the k-means clustering reached a dead end, there is something else we could do to dig into possible country differences. Principal Component Analysis (PCA) offers a way to take multiple variables and reduce such complexity to a handful of principal components (PC) that explain most of the variance in the data. In our case, we could take the variation in budget expenditure across all 10 items, and reduce them to something smaller, perhaps identifying trends where two or more items seem to change together.

After applying PCA to our dataset, we will show the additional amount of variance each new PC explains.

In [30]:
#reduce dimensions using PCA and calculate explained variance
pca = PCA()
pca.fit(df_budget_pivot)
explained_variance_ratio = pca.explained_variance_ratio_ #explained variance ratio for each principal component
cumulative_explained_variance = explained_variance_ratio.cumsum() #cumulative explained variance
n_pc_df = pd.DataFrame({'n_pc': list(range(1, 11)), 'expvar': explained_variance_ratio, 'cumexpvar': cumulative_explained_variance}) #add all lists as columns of a DataFrame

#display the DataFrame
display(n_pc_df)

,n_pc,expvar,cumexpvar
0,1,0.48,0.48
1,2,0.20,0.67
2,3,0.14,0.82
3,4,0.09,0.91
4,5,0.04,0.96
5,6,0.03,0.99
6,7,0.01,0.99
7,8,0.01,1.00
8,9,0.00,1.00
9,10,0.00,1.00


In the DataFrame above, the `n_pc` column labels the different principal components (PC), starting by that explaining the most variance, then the second one, then the third, and so on. By looking at the `expvar` column, the first PC explaines 48% of the variance, the second 20%, the third 14%, and each successive PC explains less variance, up to a point in which adding a ninth PC does not make any difference. We can also take a look at `cumexpvar` to see the cumulative explained variance given a specific number of PCs. To see it graphically, let's use the function we defined earlier to plot dot and line figures.

In [31]:
#plot the figure
x_col = 'n_pc' 
y_col = 'cumexpvar' 
x_title = 'Number of principal components'
y_title = 'Cumulative explained variance'
title = 'Explained variance according to number of principal components'

hover_text = [f'{x_title}: {x_value:.2f}<br>{y_title}: {y_value:.2f}'
              for x_value, y_value in zip(n_pc_df[x_col], n_pc_df[y_col])]

plot_line_and_scatter(n_pc_df, x_col, y_col, x_title, y_title, hover_text, title)

The figure clearly shows how each successive PC explains less variance in the data until there is a saturation. With just two PC we can already explain almost 70% of the variance, whereas with three we go above 80%. This is quite good, because just a handful of PCs capturing most of the variance will allow for a meaningful, low-dimensional visualization. Since more than three dimensions are difficult to grasp visually, we will create a DataFrame where each row will contain a country's values for the three main PCs. To try to understand more about the unsuccessful clustering process we undertook earlier, we will also add a column with the cluster each country was assigned to.

In [32]:
#run PCA with 3 PCs and turn results into DataFrame
n_pc = 3 #specify number of PCs
pca_3d = PCA(n_components=n_pc) #set PCA with number of PCs
fitted_3d_pca = pca_3d.fit_transform(df_budget_pivot) #fit PCA
pca_df = pd.DataFrame(fitted_3d_pca, columns=['PC1', 'PC2', 'PC3']) #create DataFrame from fit

#apply k-means clustering so we can then have a column with the cluster label
optimal_k = 2 #optimal number of clusters according to the silhouette method
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(df_budget_pivot)  #fit
pca_df['cluster'] = clusters #append to pca_df

#merge pca_df with DataFrame with unique countries to get country names back
unique_countries_df = pd.DataFrame(unique_countries, columns=['country']) #convert unique_countries to a DataFrame
pca_df['country'] = unique_countries_df['country'] #append 'country' column to pca_df

#display the DataFrame
pca_df


,PC1,PC2,PC3,cluster,country
0,-11.57,0.50,5.56,1,Australia
1,-12.84,5.82,-5.00,1,Austria
2,14.32,-4.48,-2.90,0,Bangladesh
3,-8.16,1.56,-2.16,1,Belgium
4,-4.28,0.50,-4.72,1,Bulgaria
...,...,...,...,...,...
50,3.18,-4.89,-7.79,0,Thailand
51,1.23,-12.56,-2.00,0,Ukraine
52,-19.50,3.15,-2.58,1,United Kingdom
53,20.71,-19.27,-0.36,0,Yemen


Perfect. The columns of the DataFrame above are quite self-explanatory. Let's start by using it for a simple two-dimensional scatterplot where the x and y axes will be taken by the two main PCs, and each dot will be a country. The dots will be colored according to the cluster each country belongs to.

In [33]:
#define custom color palette for the clusters
cluster_palette = ['#FF5733','#6A0DAD'] 

#create a trace for each cluster
trace_list = []
for cluster, color in zip(sorted(pca_df['cluster'].unique()), cluster_palette):
    cluster_data = pca_df[pca_df['cluster'] == cluster]
    
    #create hover text with country names and PCA scores
    hover_text = [f'{country}<br>PC1: {pc1:.2f}<br>PC2: {pc2:.2f}' 
                  for country, pc1, pc2 in zip(cluster_data['country'], cluster_data['PC1'], cluster_data['PC2'])]
    
    trace = go.Scatter(
        x=cluster_data['PC1'],
        y=cluster_data['PC2'],
        mode='markers',
        marker=dict(size=10, color=color, line=dict(color='white', width=1)),
        name=f'cluster {cluster}',
        hovertemplate='%{text}<extra></extra>', #<extra></extra> makes sure no group info by hoverbox is shown
        text=hover_text,
        showlegend=False
    )
    trace_list.append(trace)

#create the figure
fig = go.Figure(trace_list, layout=mylayout)

#customize layout
fig.update_layout(
    width=plot_size_shortwidth,
    title='Country values along the two main budget expenditure-derived principal components',
    xaxis=dict(title='PC 1'),
    yaxis=dict(title='PC 2'),
    hovermode='closest'
)

#show the figure
fig.show()

PC 1 has a wider range of values than PC 2. Although the the two assigned clusters do not form clearly separated dot clouds, they seeem to be cut off along the PC 1 axis. Thus, both k-meanns clustering and PCA reduced complexity in a similar way. When taking a look at how countries lay along PC 1, we can be reminded of what some development indicators would reveal: on one end we have a bunch of Western European countries and Japan, on the other end we have some African Sub-Saharan countries, and towards the middle we have a melting pot of countries straddling Eastern Europe, Latin America, the Middle East, Asia and the Pacific Islands. A pattern is more difficult to pick up for PC 2, with more crowding and some developed and in development countries both at the low range of the spectrum, like for instance Israel and Yemen.

Because we already have the data, let's add a third dimension for our visualization, corresponding to our third PC in terms of explained variance.

In [34]:
#create a trace for each cluster
trace_list = []
for cluster, color in zip(sorted(pca_df['cluster'].unique()), cluster_palette):
    cluster_data = pca_df[pca_df['cluster'] == cluster]
    
    #create hover text with country names and PCA scores
    hover_text = [f'{country}<br>PC1: {pc1:.2f}<br>PC2: {pc2:.2f}<br>PC3: {pc3:.2f}' 
                  for country, pc1, pc2, pc3 in zip(cluster_data['country'], cluster_data['PC1'], cluster_data['PC2'], cluster_data['PC3'])]
    
    trace = go.Scatter3d(
        x=cluster_data['PC1'],
        y=cluster_data['PC2'],
        z=cluster_data['PC3'],
        mode='markers',
        hoverinfo='skip',  # Skip default hover info
        text=hover_text,
        hovertemplate='%{text}<extra></extra>',  # Use the hover text without extra info
        marker=dict(size=5, color=color, line=dict(color='white', width=0.5)),
        showlegend=False
    )
    trace_list.append(trace)

#create the figure
fig = go.Figure(data=trace_list, layout=mylayout)

#customize layout
fig.update_layout(
    width=plot_size_shortwidth,
    title='Country values along the three main budget expenditure-derived principal components',
    margin=dict(l=20, r=20, t=40, b=20),
    scene=dict(
        xaxis=dict(title='PC 1', showspikes=True, spikethickness=2, spikecolor="black",
                   backgroundcolor='white', gridcolor='lightgrey', range=[-25, None]),
        yaxis=dict(title='PC 2', showspikes=True, spikethickness=2, spikecolor="black",
                   backgroundcolor='white', gridcolor='lightgrey'),
        zaxis=dict(title='PC 3', showspikes=True, spikethickness=2, spikecolor="black",
                   backgroundcolor='white', gridcolor='lightgrey')
    )
)

fig.show()

We can change the perspective by clicking on the plot and dragging. A priori, the third PC does not seem to reveal any intuitive insights. But there is something else that may throw some light into our PCs. To understand how the complexity was reduced, we need to know how the different items are related to each PC, how strongly, and in which direction. There is one metric that conveys precisely this: loadings. For each item and PC, we can obtain its loading, with its size describing how much that item corresponds to the particular PC, and its sign (positive or negative) indicating whether it is positively or negatively correlated to it. This calculation is pretty straightforward.

In [35]:
#fit PCA
pca = PCA(n_components=n_pc)
pca.fit(df_budget_pivot)

#get the loadings
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

#create a DataFrame for the loadings
loadings_df = pd.DataFrame(loadings, columns=['PC1', 'PC2', 'PC3'], index=df_budget_pivot.columns)
loadings_df = loadings_df.loc[unique_items] #ordered according to unique_items

#display the DataFrame
print(loadings_df)

                                   PC1   PC2   PC3
item                                              
General public services           7.84  3.89 -0.04
Defence                           1.90 -3.44  0.26
Public order and safety           0.68 -1.89 -0.83
Economic affairs                  2.64  2.53 -1.54
Environment protection           -0.46  0.32 -0.07
Housing and community amenities   0.20 -0.27 -0.38
Health                           -9.28  2.92 -2.45
Recreation, culture and religion -0.65  0.09  0.16
Education                         0.04 -4.76 -1.36
Social protection                -2.90  0.63  6.25


While this DataFrame contains all the information we need, it is still quite tedious to interpret. So let's take it and make a heatmap plot where each loading is in a cell, with its color telling us about both size and direction of the loading.

In [36]:
#define custom colorscale
custom_colorscale = [
    [0.0, '#8c510a'], #start color = brown
    [0.5, '#ffffff'], #midpoint color = white
    [1.0, '#01665e']  #end color = green
]

#create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=loadings_df.values,
    x=loadings_df.columns,
    y=loadings_df.index,
    colorscale=custom_colorscale,
    zmid=0,
    showscale=True,
    hoverinfo='none' ) )

#add annotations with the value of each loading
annotations = []
for i, row in enumerate(loadings_df.values):
    for j, value in enumerate(row):
        annotations.append(
            go.layout.Annotation(
                text=str(np.round(value, 2)),
                x=loadings_df.columns[j],
                y=loadings_df.index[i],
                showarrow=False,
                font=dict(color='black')
            )
        )

#customize layout
fig.update_layout(
    title='Budget expenditure-derived PCA loadings',
    annotations=annotations,
    width=plot_size_shortwidth,
    height=plot_size_shortheight,
    yaxis=dict(autorange='reversed')
)

#show the figure
fig.show()

That's much better. Each row is a different item, and each column a different PC. Starting by the leftmost column, corresponding to PC 1, we can see that general public services and health are the most correlated to the PC. However, they are so in opposite directions, such that countries with a proportionally higher expenditure on general public services tend to spend less on health, and the other way round. Looking at smaller effects, a bigger expenditure on general public services seems to be accompanied by more money for economic affairs and defence, and more investment in health goes alongside more spending on social protection. An observation we have made upon looking at our previous PCA figures is that countries with lower PC 1 values tended to belong to what we call the "First World". A hypothesis we could make, then, is that perhaps developed countries can be characterized by higher spending on health and social protection, while less developed (and almost always poorer) countries have to dedicate a bigger share of their resources to running the bureaucratic part of the administration, which is what general public services encompasses. As for the negative relationship between economic affairs and those items associated with development, we could hypothesize that less developed countries may have to invest more money into their less mature and independent economic sector.

The loadings of PC 2 tell another story, with general public services and health, along with economic affairs, going in the same direction. Although this seems to go against what we found for PC 1, it taps on a different part of the data variance. Investment on the aforementioned areas is negatively related to investment on education and defence. 

Finally, the main item driving PC 3 is social protection. Opposite to such item run health, economic affairs and education, all holding a moderately negative relationship with the PC.

## 5. Correlating budget-derived principal components with other datasets' measures

So far we have made a few conjectures with regards to the relationship among items within our PCs. Particularly, we have hypothesized that the general public services - health trade-off within PC 1 may be related to development. In what follows we will mainly try to see whether lower values of PC 1 are indeed related to development, as measured by the Human Developed Index. We will then explore the data for an example country that does not adjust to our hypotheses. Finally, since general public services played a central role in PC 1, we will look for correlations between this PC and indicators of public sector size.

### 5.1. Correlation between budget-derived principal components and Human Development Indices
Human Developed Index (HDI) is the most common measure of development. It calculated by taking into account life expectancy, education (mean years of schooling completed and expected years of schooling upon entering the education system) and per capita income. An alternative calculation, which also considers inequality, is the Inequality-adjusted HDI (IHDI). The present section will make use of a dataset from the [United Nations Reports website](https://hdr.undp.org/inequality-adjusted-human-development-index#/indicies/IHDI) containing the HDI and IHDI scores for multiple countries. After loading and processing the data, we will append it to our PCA dataset and try to correlate our different PCs to both HDI and IHDI.

The first step is reading the .xlsx file, taking only the parts of the spreadsheet containing the relevant data.

In [37]:
#read the relevant columns and rows from the Excel file
hdi_df = pd.read_excel(
    'HDR23-24_Statistical_Annex_I-HDI_Table.xlsx', 
    usecols=[1, 2, 4], 
    skiprows=5,  
    nrows=200 
)

#rename the columns
hdi_df.columns = ['country', 'HDI', 'IHDI']

#display the DataFrame
display(hdi_df)

,country,HDI,IHDI
0,Switzerland,0.97,0.89
1,Norway,0.97,0.90
2,Iceland,0.96,0.91
3,"Hong Kong, China (SAR)",0.96,0.84
4,Denmark,0.95,0.90
...,...,...,...
194,South Sudan,0.38,0.22
195,Somalia,0.38,..
196,Other countries or territories,NaN,NaN
197,Korea (Democratic People's Rep. of),..,..


We now have a clean DataFrame with three columns, one labeling the country and one each for HDI and IHDI values. The next step is trying to match the country names in this DataFrame with those of our PCA DataFrame. we will initially obtain the names in the latter that do not appear on the former.

In [38]:
#extract the country names from both dataframes
countries_pca = set(pca_df['country'])
countries_hdi = set(hdi_df['country'])

#display the country names that are not in df_hdi
display(countries_pca - countries_hdi)

{'Eswatini', 'Iran', 'Russia', 'South Korea', 'Tanzania'}

Okay, not so many. The names for these countries probably differ across datasets. We can take a look at the different country names in the HDI dataset, identify their spelling, and change it.

In [39]:
#display the DataFrame
display(countries_hdi)

{'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo (Democratic Republic of the)',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini (Kingdom of)',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guin

In [40]:
#replace countries that are written differently in hdi_df compared with pca_df
hdi_df['country'] = hdi_df['country'].replace('Eswatini (Kingdom of)', 'Eswatini')
hdi_df['country'] = hdi_df['country'].replace('Iran (Islamic Republic of)', 'Iran')
hdi_df['country'] = hdi_df['country'].replace('Russian Federation', 'Russia')
hdi_df['country'] = hdi_df['country'].replace('Korea (Republic of)', 'South Korea')
hdi_df['country'] = hdi_df['country'].replace('Tanzania (United Republic of)', 'Tanzania')

This should do it. To make sure, we will repeat the process above and see which country names differ between datasets.

In [41]:
#extract unique country names from hdi_df
countries_hdi = set(hdi_df['country'])

#print the countries that are not in countries_pca
display(countries_pca - countries_hdi)

set()

An empty set means that no names differ. Good. Time to merge both DataFrames.

In [42]:
#merge both DataFrames
pca_df = pd.merge(pca_df, hdi_df, how='left', on='country')

#display the merged DataFrame
display(pca_df)

,PC1,PC2,PC3,cluster,country,HDI,IHDI
0,-11.57,0.50,5.56,1,Australia,0.95,0.86
1,-12.84,5.82,-5.00,1,Austria,0.93,0.86
2,14.32,-4.48,-2.90,0,Bangladesh,0.67,0.47
3,-8.16,1.56,-2.16,1,Belgium,0.94,0.88
4,-4.28,0.50,-4.72,1,Bulgaria,0.80,0.70
...,...,...,...,...,...,...,...
50,3.18,-4.89,-7.79,0,Thailand,0.80,0.68
51,1.23,-12.56,-2.00,0,Ukraine,0.73,0.68
52,-19.50,3.15,-2.58,1,United Kingdom,0.94,0.86
53,20.71,-19.27,-0.36,0,Yemen,0.42,0.28


We now have a DataFrame where, for each country, we have the PC values derived from our analyses, on one hand, and the HDI and IHDI we just obtained, on the other. Even if our hypothesis linked HDI only to PC 1, we may as well run correlations between each PC and each version of the HDI.  

In [43]:
#calculate the correlations with p-values
corr_pc1_hdi, pvalue_pc1_hdi = stats.pearsonr(pca_df['PC1'], pca_df['HDI'])
corr_pc2_hdi, pvalue_pc2_hdi = stats.pearsonr(pca_df['PC2'], pca_df['HDI'])
corr_pc3_hdi, pvalue_pc3_hdi = stats.pearsonr(pca_df['PC3'], pca_df['HDI'])

corr_pc1_ihdi, pvalue_pc1_ihdi = stats.pearsonr(pca_df['PC1'], pca_df['IHDI'])
corr_pc2_ihdi, pvalue_pc2_ihdi = stats.pearsonr(pca_df['PC2'], pca_df['IHDI'])
corr_pc3_ihdi, pvalue_pc3_ihdi = stats.pearsonr(pca_df['PC3'], pca_df['IHDI'])

#print the results
print(f"Correlation between PC1 and HDI, r: {corr_pc1_hdi:.2f}, p-value: {pvalue_pc1_hdi:.2e}")
print(f"Correlation between PC2 and HDI, r: {corr_pc2_hdi:.2f}, p-value: {pvalue_pc2_hdi:.2e}")
print(f"Correlation between PC3 and HDI, r: {corr_pc3_hdi:.2f}, p-value: {pvalue_pc3_hdi:.2e}")

print(f"Correlation between PC1 and IHDI, r: {corr_pc1_ihdi:.2f}, p-value: {pvalue_pc1_ihdi:.2e}")
print(f"Correlation between PC2 and IHDI, r: {corr_pc2_ihdi:.2f}, p-value: {pvalue_pc2_ihdi:.2e}")
print(f"Correlation between PC3 and IHDI, r: {corr_pc3_ihdi:.2f}, p-value: {pvalue_pc3_ihdi:.2e}")

Correlation between PC1 and HDI, r: -0.83, p-value: 2.89e-15
Correlation between PC2 and HDI, r: 0.18, p-value: 1.89e-01
Correlation between PC3 and HDI, r: 0.14, p-value: 3.23e-01
Correlation between PC1 and IHDI, r: -0.83, p-value: 7.22e-15
Correlation between PC2 and IHDI, r: 0.19, p-value: 1.67e-01
Correlation between PC3 and IHDI, r: 0.09, p-value: 5.13e-01


The correlation between PC1 and either HDI or IHDI is very high. The *r* (correlation coefficient) is approximately -0.83 for both, which translates into an *r<sup>2</sup>* of -0.699. This means that around a whopping 70% of the variation in the data can be explained by the correlation, indeed establishing a relationship between PC1 and HDI. Its direction is as expected: countries with lower PC1 values, investing more on health and social protection and less on general public services and economic affairs, are generally more developed.

Correlations involving other PCs yield lower coefficients and, although consistently positive, their *p-values* are not lower than 0.05, the usually accepted threshold for statistical significance.

We can next visualize the correlation between PC 1 and HDI (or IHDI) in order to appreciate the position of each country in such relationship. Since this is not the last correlation figure we will compose, we can define a function to automatize their creation.

In [44]:
#define function to create correlation plot
def plot_correlation(df, x_col, y_col, x_title, y_title, hover_text):
    
    #turn columns to numeric
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')

    #drop rows with NaN values
    df = df.dropna(subset=[x_col, y_col])

    #create scatter plot
    scatter = go.Scatter(
        x=df[x_col],
        y=df[y_col],
        mode='markers',
        name='Data points',
        hoverinfo='text',
        text=hover_text,
        showlegend=False, 
        marker=dict(size=10, color='black', line=dict(color='white', width=1)),
    )

    #calculate the line of best fit
    fit = np.polyfit(df[x_col], df[y_col], deg=1)
    fit_fn = np.poly1d(fit)

    #create line plot
    line = go.Scatter(
        x=df[x_col],
        y=fit_fn(df[x_col]),
        mode='lines',
        name=None,
        hoverinfo='none',
        showlegend=False, 
        line=dict(color='red')
    )

    #create the figure
    fig = go.Figure(data=[scatter, line], layout=mylayout)

    #customize layout
    fig.update_layout(
        width=plot_size_shortwidth,
        height=plot_size_shortheight,
        title=f'Correlation, {x_title} and {y_title}',
        xaxis_title=x_title,
        yaxis_title=y_title,
        margin=dict(l=40, r=40, t=40, b=40)
    )

    #show the figure
    fig.show()

#plotting correlation between PC 1 and HDI
x_col = 'PC1' #name of column for x-axis
y_col = 'HDI' #name of column for y-axis
x_title = 'Principal Component 1' #title for x-axis
y_title = 'Human Development Index' #title for y-axis

hover_text = [f'{country}<br>{x_title}: {x_value:.2f}<br>{y_title}: {y_value:.2f}' 
              for country, x_value, y_value in zip(pca_df['country'], pca_df[x_col], pca_df[y_col])] #defining structure of hover text

plot_correlation(pca_df, x_col, y_col, x_title, y_title, hover_text)

Beautiful. One of the advantages of having made this plot is that we can appreciate which countries are closest to the line of best fit and which are furthest away from it. One case drawing our attention is that of Switzerland, with a very high HDI (as of 2024, actually first in the world) but with a middle-range value in PC 1. What is going on in here? we will sidetrack a bit to find out.

### 5.2. Exploring outliers: the case of Switzerland

If low PC 1 was univocally related to high HDI, we should expect Switzerland to have one of the lowest PC 1 values. However, that is not the case. Is there anything different in the way Switzerland spends its budget compared to other highly developed countries? A way to look into that is to compare Switzerland to an average of those countries scoring the lowest in PC 1. Let's do that by first obtaining a DataFrame where the budget percentage for each item is present for Switzerland and for the average of those 10 countries scoring the lowest in PCA 1.

In [45]:
#create a DataFrame with the average budget percentages for the 10 countries scoring the lowest in PC1
bottom10_PCA1_countries = pca_df.sort_values(by='PC1', ascending=True).head(10)['country'].tolist() #list with the bottom 10 countries
bottom10_PCA_df = df_budget[df_budget['country'].isin(bottom10_PCA1_countries)] #select part of the PCA DataFrame corresponding to those countries
bottom10_PCA_sum_df = bottom10_PCA_df.groupby('item')['perc_budget'].mean().reset_index() #create summary DataFrame by averaging item percentages over countries
bottom10_PCA_sum_df['country'] = 'bottom 10 PC1'

#create a DataFrame with data for Switzerland
switzerland_budget_df = df_budget[df_budget['country'] == 'Switzerland']

#merge both DataFrames
bottom10_switzerland_df = pd.concat([bottom10_PCA_sum_df, switzerland_budget_df], ignore_index=True)

#display the DataFrame
display(bottom10_switzerland_df)

,item,perc_budget,country,item_code,year,value,total_budget
0,Defence,4.85,bottom 10 PC1,NaN,NaN,NaN,NaN
1,Economic affairs,8.50,bottom 10 PC1,NaN,NaN,NaN,NaN
2,Education,17.40,bottom 10 PC1,NaN,NaN,NaN,NaN
3,Environment protection,2.28,bottom 10 PC1,NaN,NaN,NaN,NaN
4,General public services,6.68,bottom 10 PC1,NaN,NaN,NaN,NaN
5,Health,35.13,bottom 10 PC1,NaN,NaN,NaN,NaN
6,Housing and community amenities,1.20,bottom 10 PC1,NaN,NaN,NaN,NaN
7,Public order and safety,6.46,bottom 10 PC1,NaN,NaN,NaN,NaN
8,"Recreation, culture and religion",2.93,bottom 10 PC1,NaN,NaN,NaN,NaN
9,Social protection,14.57,bottom 10 PC1,NaN,NaN,NaN,NaN


We will now visualize the information in this dataset by creating a plot where the percentage of the budget spent on each item is shown side by side for both our groups.

In [46]:
#create traces
traces = []
for i, item in enumerate(unique_items):
    item_df = bottom10_switzerland_df[bottom10_switzerland_df['item'] == item]
    hover_text = [f'Group: {country}<br>{item}: {perc_budget:.2f}' for country, perc_budget in zip(item_df['country'], item_df['perc_budget'])]
    trace = go.Scatter(
        x=item_df['country'],
        y=item_df['perc_budget'],
        mode='lines+markers',
        name=item,
        line=dict(color=palette_qual[i % len(palette_qual)]),
        marker=dict(color=palette_qual[i % len(palette_qual)], symbol='circle', size=10, line=dict(color='white', width=1)),
        hoverinfo='text',
        text=hover_text
    )
    traces.append(trace)

#create the figure
fig = go.Figure(data=traces, layout=mylayout)

#customize layout
fig.update_layout(
    title='Budget Allocation Comparison: Bottom 10 PC1 Countries vs. Switzerland',
    xaxis_title='',
    yaxis_title='Percentage of Budget',
    legend_title='Item',
    width=plot_size_shortwidth,
    height=plot_size_shortheight,
    margin=dict(l=50, r=50, t=50, b=50),
    legend=dict(x=1.05, y=1, xanchor='left', yanchor='top'),
    xaxis=dict(showgrid=False)
)

#show the figure
fig.show()

No wonder why Switzerland did not score high on PC 1. The percentage assigned to health is much lower than that of the bottom 10 PC 1 countries, whereas that for general public services is a bit higher. The data is striking, particularly considering that Switzerland is renowned for having one of the best healthcare systems in the world. But this mismatch between expectations and data may be a product of the nature of the Swiss healthcare system. Indeed, despite Switzerland lacking having universal healthcare, it is not freely provided by the state. Instead, citizens are required to purchase private health insurance. This healthcare externalization could make public health-related expenses be lower, or simply be indirect, considered as part of a different item. For instance, public subsidies for such private healthcare programs could be labeled as expenditure on social protection, or on general public services.  

Aside from health and general public services, the other most remarkable difference is that Switzerland seems to invest in education substantially more than other developed countries, with the percentages for other items being similar.

### 5.3. Correlation between budget-derived Principal Component 1 and public sector size
Before finishing, we will try to link our principal components to one more type of indicator. Some of the budget expenditure items we have worked with are related to sectors with plenty of public employees: government bureaucrats, doctors and nurses, public school teachers and public university professors, the military, and so on. [The World Bank Data Catalog portal includes a dataset called *Worldwide Bureaucracy Indicators*](https://datacatalog.worldbank.org/search/dataset/0038132), containing a series of indicators related to the public sector. From this dataset, we could identify the indicators most related to our items, and see whether, across countries, they correlate with our PCs.

As always, first we need to load the dataset and wrangle it a bit.

In [47]:
#load as DataFrame
df_pubdata = pd.read_csv('WWBIData.csv')

#display the DataFrame
display(df_pubdata)

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 25
0,Afghanistan,AFG,"Core Public Administration workers, as a share...",BI.EMP.FRML.CA.PB.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Core Public Administration workers, as a share...",BI.EMP.PWRK.CA.PB.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,"Core Public Administration workers, as a share...",BI.EMP.TOTL.CA.PB.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,Cross-country public sector pay comparison rat...,BI.PWK.CMPA.GE.SM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,Cross-country public sector pay comparison rat...,BI.PWK.CMPA.GE.MD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60999,Zimbabwe,ZWE,"Teachers, as a share of public formal employees",BI.EMP.FRML.TS.PB.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61000,Zimbabwe,ZWE,"Teachers, as a share of public paid employees",BI.EMP.PWRK.TS.PB.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61001,Zimbabwe,ZWE,"Teachers, as a share of public total employees",BI.EMP.TOTL.TS.PB.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61002,Zimbabwe,ZWE,Wage bill as a percentage of GDP,BI.WAG.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,6.24,2.59,1.15,0.77,4.33,7.27,11.06,12.28,12.30,13.15,12.98,13.23,10.85,7.87,3.75,6.44,NaN


For each row, we have the country name and code, the indicator name and code, and a few rows corresponding to the indicator's data for different years. We can already appreciate that data is missing for a lot of years and countries... Let's see what we can do. Next we will unify country names with those of our previous DataFrames. The process will be the same as what we did with our IDH dataset, so there is no need to explain the steps with such detail.

In [48]:
#extract the country names from both public sector indicators and budget DataFrames
countries_pubdata = set(df_pubdata['Country Name'].unique())
countries_budget = set(df_budget['country'].unique())

#display the countries that are not in df_pubdata
display(countries_budget - countries_pubdata)

{'Czechia', 'Kyrgyzstan', 'Slovakia', 'South Korea'}

In [49]:
#display the unique countries tin df_pubdata
display(countries_pubdata)

{'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Costa Rica',
 'Croatia',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Dem. Rep. Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Hond

In [50]:
#replace countries that are written differently in countries_pubdata compared with pca_df
df_pubdata['Country Name'] = df_pubdata['Country Name'].replace('Czech Republic', 'Czechia')
df_pubdata['Country Name'] = df_pubdata['Country Name'].replace('Kyrgyz Republic', 'Kyrgyzstan')
df_pubdata['Country Name'] = df_pubdata['Country Name'].replace('Slovak Republic', 'Slovakia')
df_pubdata['Country Name'] = df_pubdata['Country Name'].replace('Korea', 'South Korea')

#extract the country names from df_pubdata
countries_pubdata = set(df_pubdata['Country Name'].unique())

#display the countries that are not in df_pubdata
display(countries_budget - countries_pubdata)

set()

Again, an empty set means that we got rid of any discrepancies in country names. Now we only have to select the rows of our new DataFrame that belong to countries present in our analyses.

In [51]:
#select only rows for countries present in our analyses
df_pubdata = df_pubdata[df_pubdata['Country Name'].isin(countries_budget)]

And now we get to the interesting part. We can take a look at the different indicators and choose the most relevant ones.

In [52]:
#display unique indicator names
df_pubdata['Indicator Name'].unique()

array(['Core Public Administration workers, as a share of public formal employees',
       'Core Public Administration workers, as a share of public paid employees',
       'Core Public Administration workers, as a share of public total employees',
       'Cross-country public sector pay comparison ratio, by occupation: Government economist (using mean)',
       'Cross-country public sector pay comparison ratio, by occupation: Government economist (using median)',
       'Cross-country public sector pay comparison ratio, by occupation: Hospital doctor (using mean)',
       'Cross-country public sector pay comparison ratio, by occupation: Hospital doctor (using median)',
       'Cross-country public sector pay comparison ratio, by occupation: Hospital nurse (using mean)',
       'Cross-country public sector pay comparison ratio, by occupation: Hospital nurse (using median)',
       'Cross-country public sector pay comparison ratio, by occupation: Judge (using mean)',
       'Cross-count

Among such extensive list, a few look promising. Let's explain why.

*Core Public Administration workers, as a share of public total employees*: this refers to the number of public employess that keep a country's agencies running over a the total of public employees, which also involve health, education, defence, etc... This measure could correlate positively with the percentage of the budget dedicated to general public services, and consequently also correlate positively with PC 1. 

*Health workers, as a share of public total employees*: this could correlate negatively with PC 1, as lower PC 1 values are related to more investment in health.

*Medical workers, as a share of public total employees*: same as indicator above.

*Public Administration workers, as a share of public total employees*: same as first indicator of interest.

*Public sector employment, as a share of total employment*: this is a bit tricky, since it informs us of the amount of public workers over the total workforce. These workers include both those in the core administration, but also health and education professionals. So it is not clear how they should correlate with our PCs, and in particular with PC 1.

Having chosen our indicators, we can now filter the dataset to only include the rows concerning them.

In [53]:
#create a list with our selected indicators
selected_indicators = [
    'Core Public Administration workers, as a share of public total employees',
    'Health workers, as a share of public total employees',
    'Medical workers, as a share of public total employees',
    'Public Administration workers, as a share of public total employees',
    'Public sector employment, as a share of total employment'
]

#filter the DataFrame to include only the selected indicators
df_pubdata = df_pubdata[df_pubdata['Indicator Name'].isin(selected_indicators)]

Next, we can see if the data available for each of these indicators is enough to run our correlations. Since we do not want to work with data that is too old, we will stick to the same year range as before, and choose only values corresponding to years from 2010 to 2020 (this being the last available year). Within this range, we will keep the most recent data we have for each country and indicator.

In [54]:
#define years of interest
years = list(map(str, range(2010, 2020 + 1)))

#define function to find the most recent year with data
def find_most_recent_year(row):
    for year in reversed(years):
        if not pd.isna(row[year]):
            return year
    return None

#iterate over each country and indicator and run function find_most_recent_year
most_recent_year_results = [] #create empty set to populate while iterating

for (country, indicator), group in df_pubdata.groupby(['Country Name', 'Indicator Name']):
    most_recent_year = group.apply(find_most_recent_year, axis=1).max()
    if most_recent_year and most_recent_year in group.columns:
        value = group[most_recent_year].values[0]
    else:
        most_recent_year = None
        value = None
    most_recent_year_results.append([country, indicator, most_recent_year, value])

most_recent_year_df = pd.DataFrame(most_recent_year_results, columns=['country', 'indicator', 'most_recent_year', 'value']) #create DataFrame by giving column names to results

#display the DataFrame
display(most_recent_year_df)

,country,indicator,most_recent_year,value
0,Australia,"Core Public Administration workers, as a share...",None,NaN
1,Australia,"Health workers, as a share of public total emp...",2015,0.26
2,Australia,"Medical workers, as a share of public total em...",2015,0.11
3,Australia,"Public Administration workers, as a share of p...",2015,0.26
4,Australia,"Public sector employment, as a share of total ...",2015,0.20
...,...,...,...,...
270,Zambia,"Core Public Administration workers, as a share...",2015,0.03
271,Zambia,"Health workers, as a share of public total emp...",2015,0.04
272,Zambia,"Medical workers, as a share of public total em...",None,NaN
273,Zambia,"Public Administration workers, as a share of p...",2015,0.05


For each country and indicator, `most_recent_year` tells us the msot recent year for which we have data, and `value` gives us the value for this most recent indicator. If there is no data within the specified years range, the aforementioned columns will encode it as *None* and *NaN*. Now, irrespective of the year, we can know the number of countries for which we have data within each indicator. 

In [55]:
#create DataFrame with rows where 'value' is not NaN
available_data_df = most_recent_year_df[most_recent_year_df['value'].notna()]

#count and display the number of rows
available_data_df['indicator'].value_counts()

indicator
Public sector employment, as a share of total employment                    45
Health workers, as a share of public total employees                        12
Public Administration workers, as a share of public total employees         12
Medical workers, as a share of public total employees                        7
Core Public Administration workers, as a share of public total employees     6
Name: count, dtype: int64

Bummer... For most indicators, we only have a handful of countries. The only indicator where our sample is decent enough is *Public sector employment, as a share of total employment*. This was also our trickiest variable, but it is the only one we can really work with, so we may as well go ahead with it. Let's get the data for that indicator ready and attach it to our PCA DataFrame.

In [56]:
#get a DataFrame with only data for our indicator of interest
pubsecsize_df = most_recent_year_df[most_recent_year_df['indicator'] == 'Public sector employment, as a share of total employment'] #filter to include only rows of our indicator of interest
pubsecsize_df = pubsecsize_df[["country", "value"]] #select only necessary columns
pubsecsize_df = pubsecsize_df.set_axis(['country', 'public_over_total_employees'], axis=1) #rename columns

#merge PCA and public sector size DataFrames
pca_df = pd.merge(pca_df, pubsecsize_df, how='left', on='country')

The only step before running the correlations is to get rid of NaN values, since it is a requirement to use the *pearsonr* function from the *scipy.stats* library. 

In [57]:
#drop rows with NaN values in the relevant columns, since the pearson correlation function we use cannot handle NaN
pca_df_nona = pca_df[['PC1', 'PC2', 'PC3', 'public_over_total_employees']].dropna()

#calculate the correlations with p-values
corr_pc1_pubsec, pvalue_pc1_pubsec = stats.pearsonr(pca_df_nona['PC1'], pca_df_nona['public_over_total_employees'])
corr_pc2_pubsec, pvalue_pc2_pubsec = stats.pearsonr(pca_df_nona['PC2'], pca_df_nona['public_over_total_employees'])
corr_pc3_pubsec, pvalue_pc3_pubsec = stats.pearsonr(pca_df_nona['PC3'], pca_df_nona['public_over_total_employees'])

#print the results
print(f"Correlation between PC1 and proportion of public sector employment, as a share of total employment, r: {corr_pc1_pubsec:.2f}, p-value: {pvalue_pc1_pubsec:.2e}")
print(f"Correlation between PC2 and proportion of public sector employment, as a share of total employment, r: {corr_pc2_pubsec:.2f}, p-value: {pvalue_pc2_pubsec:.2e}")
print(f"Correlation between PC3 and proportion of public sector employment, as a share of total employment, r: {corr_pc3_pubsec:.2f}, p-value: {pvalue_pc3_pubsec:.2e}")

Correlation between PC1 and proportion of public sector employment, as a share of total employment, r: -0.48, p-value: 8.29e-04
Correlation between PC2 and proportion of public sector employment, as a share of total employment, r: 0.01, p-value: 9.69e-01
Correlation between PC3 and proportion of public sector employment, as a share of total employment, r: 0.10, p-value: 5.10e-01


Only PC 1 holds a significant correlation with the proportion of public employees over the whole workforce, and it is a negative correlation. That means that countries scoring lower in PC 1, which tend to be more developed, are also characterized by a relatively bigger public workforce. We can use our previous function to easily plot such correlation.

In [58]:
#plotting correlation between PC 1 and proportion of public sector employment
x_col = 'PC1'
y_col = 'public_over_total_employees'
x_title = 'Principal Component 1'
y_title = 'Prop. public employees over total workforce'

hover_text = [f'{country}<br>{x_title}: {x_value:.2f}<br>{y_title}: {y_value:.2f}' 
              for country, x_value, y_value in zip(pca_df['country'], pca_df[x_col], pca_df[y_col])]

plot_correlation(pca_df, x_col, y_col, x_title, y_title, hover_text)

Compared to our HDI correlation, here we see that more countries stray away from the line of best fit. In particular, countries like Mongolia, Slovakia, Israel and Latvia are along the middle in PC 1, yet they have a bigger public workforce. In any case, it looks like there is a tendency for more developed countries to have more public employees, probably because they can afford them.

## 6. Conclusion
Our project asked whether we could categorize countries based on how they spend their budget over a series of items, and whether this could help us predict some socio-economic indicators. By using k-means clustering, we found that countries did not form clear, separate clusters. However, through Principal Component Analysis we could take the multiple expenditure items and reduce them to a handful of Principal Components, the most important of which succesfully predicted such indicators. In particular, our main PC captured a trade-off between spending on health and social protection, on one hand, and on general public services and economic affairs, on the other. Countries spending more on the former tended to have a higher HDI and a bigger share of public employees over the total workforce. Thus, although clustering was not the ideal technique for our data, a more continuum-based approach allowed us to relate budget expenditure and socio-economic indicators.

While our project extracted meaningful insights, the data we worked with presented a series of challenges that may have also influenced our results. The biggest limitation is the fact that the available data, both for budget expenditure and public sector indicators, came mostly from developed countries. Had data been available for all countries, clustering may have proved more successful.

Another problem may have been different countries assigning the same type of spending to different items. For instance, a rent subsidy may be either classified under housing and community amenities or under social protection. This may have caused the presence of some outliers, like the huge spending of Iran on social protection, which had an impact on that country's PC 3 value. Moreover, we also had to exclude the cryptic *other* category. Under closer inspection, a lot of the spending grouped as part of this item may have been recategorized to other items, but unfortunately we had not access to such fine-grained data.

All and all, this project is an example that the nature of the available data greatly conditions which techniques we can use to achieve our goals. 